In [1]:
# import packages

from mattertune import configs as MC
import mattertune as mt
from pathlib import Path

In [2]:
# load orb-v2 model

config = MC.ORBBackboneConfig(
    pretrained_model="orb-v2",
    system=MC.ORBSystemConfig(
        radius=10.0,
        max_num_neighbors=20
    ),
    properties=[
        MC.EnergyPropertyConfig(
            loss=MC.MAELossConfig(),
            loss_coefficient=1.0
        ),
        MC.ForcesPropertyConfig(
            loss=MC.MAELossConfig(),
            loss_coefficient=10.0,
            conservative=False
        ),
    ],
    optimizer=MC.AdamWConfig(lr=1e-4),
)

model = config.create_model()

/opt/anaconda3/envs/orb-partitioning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/orb-partitioning/lib/python3.10/site-packages/orb_models/forcefield/pretrained.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi

In [3]:
# imports for calculation
from ase.optimize import BFGS
from ase import Atoms
from ase.visualize import view
from ase.io import read

In [4]:
# create the calculator
calculator = model.ase_calculator()

In [5]:
atoms = read("H2O.xyz")
atoms.calc = calculator

In [6]:
# Calculate potential energy
print("Final energy:", atoms.get_potential_energy())

You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(barebones=True)` started running. The progress bar is disabled so you might want to manually print the progress in your model.
/opt/anaconda3/envs/orb-partitioning/lib/python3.10/site-pa

Final energy: -0.005859195254743099


# Partitioning

In [7]:
from ase import Atoms

## Functions

The `atoms_to_graph` function takes in an `ase.Atoms` object and outputs a networkx graph. Edges are created between two nodes if they are within the natural cutoffs provided.

In [8]:
import networkx as nx
from ase.neighborlist import NeighborList, natural_cutoffs

def atoms_to_graph(atoms: Atoms):
    G = nx.Graph()

    G.add_nodes_from(range(len(atoms)))

    # Create graph

    # Generate radial cutoffs (https://wiki.fysik.dtu.dk/ase/ase/neighborlist.html#ase.neighborlist.natural_cutoffs)
    cutoffs = natural_cutoffs(atoms)

    # Create the neighborlist and update with atoms
    neighbor_list = NeighborList(cutoffs, self_interaction=False, bothways=True)
    neighbor_list.update(atoms)

    # Create appropriate edges in G
    for u in range(len(atoms)):
        neighbors, offsets = neighbor_list.get_neighbors(u)
        for v in neighbors:
            dist = ((atoms.positions[u] - atoms.positions[v])**2).sum()**0.5
            G.add_edge(u, v, weight=dist)

    return G

The `partition_graph` function takes a graph and partitions it using METIS

In [9]:
import metis

def partition_graph(G: nx.Graph, num_partitions: int = 2):
    edgecuts, parts = metis.part_graph(G, num_partitions)
    return parts

The `partitions_to_atoms` function takes the original Atoms object, the partitions from a partitioning algorithm, and creates separate Atoms objects containing the atoms from each partition

In [10]:
def partitions_to_atoms(atoms: Atoms, partitions: list):
    num_partitions = len(set(partitions))

    # Right now I will only restore the atomic numbers and positions
    numbers = [[] for _ in range(num_partitions)]
    positions = [[] for _ in range(num_partitions)]

    for i in range(len(atoms)):
        number = atoms.get_atomic_numbers()[i]
        position = atoms.get_positions()[i]

        partition = partitions[i]

        numbers[partition].append(number)
        positions[partition].append(position)

    return [Atoms(numbers=numbers[i], positions=positions[i]) for i in range(num_partitions)]

## Step-by-step partitioning and inference

In [11]:
from ase.io import read

# Read atoms
atoms = Atoms('H2O',
             positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]],
             cell=[10, 10, 10],
             pbc=True)

# Convert to graph
G = atoms_to_graph(atoms)

Partitioning

In [12]:
partitions = partition_graph(G)
len(set(partitions))

1

We can optionally visualize the partition (this might be really laggy)

In [13]:
visualize = False

if visualize:
    import py3Dmol

    positions = atoms.get_positions()
    colors = ["red", "blue", "green"]

    view = py3Dmol.view(width=800, height=600)

    for i, pos in enumerate(positions):
        view.addSphere({
            'center': {'x': pos[0], 'y': pos[1], 'z': pos[2]},
            'radius': 0.5,
            'color': colors[partitions[i]]
        })

    view.show()


Now we take the partitioned atoms and convert them back into their own `ase.Atoms` objects

In [14]:
partitioned_atoms = partitions_to_atoms(atoms, partitions)

IndexError: list index out of range

Now we can run inference on each of the atoms using mattertune

In [57]:
atoms.calc = calculator

for part in partitioned_atoms:
    part.calc = calculator

total_potential = atoms.get_potential_energy()

partition_potentials = []

for i, part in enumerate(partitioned_atoms):
    partition_potentials.append(part.get_potential_energy())

You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(barebones=True)` started running. The progress bar is disabled so you might want to manually print the progress in your model.
You are running in `Trainer(barebones=True)` mode. All feat

Print results

In [58]:
print(f"Potential energy of all atoms:", total_potential)
print(f"Sum of potential energies across partitions", sum(partition_potentials))

for i, pe in enumerate(partition_potentials):
    print(f"Potential energy of Partiton #{i}:", pe)

Potential energy of all atoms: -0.014182958751916885
Sum of potential energies across partitions -0.0107995574362576
Potential energy of Partiton #0: -0.004986708052456379
Potential energy of Partiton #1: 0.0003144163638353348
Potential energy of Partiton #2: -0.006127265747636557


In [59]:
atoms.get_forces()

You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(barebones=True)` started running. The progress bar is disabled so you might want to manually print the progress in your model.


array([[-0.06008359,  0.01884918,  0.00663478],
       [-0.05677664,  0.01673258,  0.02490705],
       [-0.06625177,  0.0693715 ,  0.06599157],
       [ 0.02383402,  0.03705235,  0.03745429],
       [-0.08792569,  0.01976779,  0.06394845],
       [-0.04984496,  0.05295379,  0.0098209 ],
       [ 0.00734235,  0.08901956,  0.01151881],
       [-0.03085612, -0.03642404,  0.05945247],
       [-0.06008349,  0.01884929,  0.00663496],
       [-0.05677638,  0.01673249,  0.02490694],
       [-0.06625135,  0.06937159,  0.06599185],
       [ 0.02383389,  0.03705242,  0.03745428],
       [-0.0879261 ,  0.01976755,  0.06394785],
       [-0.04984542,  0.05295397,  0.00982062],
       [ 0.00734282,  0.0890191 ,  0.01151911],
       [-0.03085601, -0.03642388,  0.0594525 ],
       [ 0.09576453, -0.04970675, -0.07404261],
       [ 0.05001447, -0.04833011, -0.0577336 ],
       [ 0.11192495, -0.09128665, -0.07252418],
       [ 0.06285973, -0.07800005, -0.0754272 ],
       [ 0.09576452, -0.04970677, -0.074

In [60]:
partitioned_atoms[0].get_forces()

You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(barebones=True)` started running. The progress bar is disabled so you might want to manually print the progress in your model.


array([[-0.05746779,  0.00080313,  0.12661447],
       [-0.10788871, -0.02014599,  0.05303428],
       [ 0.05374903,  0.05866078, -0.03771198],
       [-0.01675874,  0.13633163, -0.08589168],
       [-0.04854517,  0.11762728,  0.13012366],
       [-0.04776288, -0.023279  , -0.03332747],
       [-0.08673859,  0.12991345,  0.1950409 ],
       [-0.15330528,  0.01722046,  0.01433091],
       [ 0.00583186,  0.03620213, -0.03271572],
       [ 0.0959549 , -0.09083721, -0.02315849],
       [ 0.12546074, -0.188864  , -0.15655266],
       [ 0.01770594, -0.01356754, -0.02894782],
       [-0.06997534,  0.2795211 , -0.14156042],
       [ 0.00067682,  0.03435623,  0.04708203],
       [-0.05347244,  0.08428054,  0.10131614],
       [-0.04653954, -0.01101781,  0.07732672],
       [-0.03427304,  0.05284649, -0.00170678],
       [-0.06692598,  0.00469148,  0.01999888],
       [-0.04072165,  0.06063529,  0.04145567],
       [ 0.04041349, -0.03222631, -0.05054692],
       [-0.04039866, -0.05009482, -0.058